In [ ]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


### Loading the data

In [ ]:

# Define constants
train_dir = 'Corn/train/'
test_dir = 'Corn/test/'
batch_size = 32
num_classes = 4
input_shape = (250, 250, 3)  # Adjust according to your image dimensions

# Function to filter images based on extensions
def filter_images_by_extension(directory):
    valid_extensions = ['.jpg', '.jpeg', '.png']
    image_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if any(file.lower().endswith(ext) for ext in valid_extensions):
                image_files.append(os.path.join(root, file))
    return image_files

# Filter image files
train_image_files = filter_images_by_extension(train_dir)
test_image_files = filter_images_by_extension(test_dir)


In [ ]:

# Create DataFrame for training and testing
train_df = pd.DataFrame({
    'filename': train_image_files,
    'class': [os.path.basename(os.path.dirname(path)) for path in train_image_files]
})

test_df = pd.DataFrame({
    'filename': test_image_files,
    'class': [os.path.basename(os.path.dirname(path)) for path in test_image_files]
})

# Data augmentation and normalization for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Normalization for testing
test_datagen = ImageDataGenerator(rescale=1./255)

# Generating batches of training and testing data
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='filename',
    y_col='class',
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical')

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='filename',
    y_col='class',
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical')


Found 7316 validated image filenames belonging to 4 classes.
Found 1829 validated image filenames belonging to 4 classes.


### Training the model

In [ ]:

# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=10, validation_data=test_generator)



Epoch 1/10
229/229 [==============================] - 184s 801ms/step - loss: 0.3889 - accuracy: 0.8472 - val_loss: 0.1793 - val_accuracy: 0.9311
Epoch 2/10
229/229 [==============================] - 203s 886ms/step - loss: 0.1620 - accuracy: 0.9374 - val_loss: 0.1556 - val_accuracy: 0.9399
Epoch 3/10
229/229 [==============================] - 205s 894ms/step - loss: 0.1420 - accuracy: 0.9478 - val_loss: 0.1272 - val_accuracy: 0.9568
Epoch 4/10
229/229 [==============================] - 207s 901ms/step - loss: 0.1185 - accuracy: 0.9554 - val_loss: 0.1200 - val_accuracy: 0.9574
Epoch 5/10
229/229 [==============================] - 203s 887ms/step - loss: 0.1288 - accuracy: 0.9549 - val_loss: 0.2056 - val_accuracy: 0.9185
Epoch 6/10
229/229 [==============================] - 202s 880ms/step - loss: 0.1156 - accuracy: 0.9535 - val_loss: 0.1092 - val_accuracy: 0.9612
Epoch 7/10
229/229 [==============================] - 210s 918ms/step - loss: 0.1041 - accuracy: 0.9615 - val_loss: 0.0942 -

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)

58/58 [==============================] - 14s 235ms/step - loss: 0.1155 - accuracy: 0.9634
Test accuracy: 0.9633679389953613


In [ ]:
import matplotlib.pyplot as plt
import cv2

### Saving the Model

In [ ]:

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the converted model to a file
with open('corn_disease.tflite', 'wb') as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: /var/folders/wc/4vhbxzyn749cv5d6p5s6nw100000gp/T/tmp7ny6gzjc/assets


INFO:tensorflow:Assets written to: /var/folders/wc/4vhbxzyn749cv5d6p5s6nw100000gp/T/tmp7ny6gzjc/assets
2024-04-19 15:00:31.223141: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-04-19 15:00:31.223154: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-04-19 15:00:31.223721: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/wc/4vhbxzyn749cv5d6p5s6nw100000gp/T/tmp7ny6gzjc
2024-04-19 15:00:31.224397: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-04-19 15:00:31.224401: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /var/folders/wc/4vhbxzyn749cv5d6p5s6nw100000gp/T/tmp7ny6gzjc
2024-04-19 15:00:31.226752: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-04-19 15:00:31.227344: I tensorflow/cc/saved_model/load

In [ ]:
model.save("corn_model.h5")

/Users/student/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### Predictions

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the image you want to predict
img_path = 'cercospora.JPG'  # Provide the path to your image
img = image.load_img(img_path, target_size=(250, 250))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array /= 255.  # Rescale pixel values to [0, 1]

# Make prediction
prediction = model.predict(img_array)
predicted_class = np.argmax(prediction)

# Map predicted class index to class label
class_labels = train_generator.class_indices
predicted_label = [k for k, v in class_labels.items() if v == predicted_class][0]

print("Predicted class:", predicted_label)
for i in range(0,4):
    prediction[0][i] = np.round((prediction[0][i]),2)
print("Prediction probabilities:", prediction)


1/1 [==============================] - 0s 58ms/step
Predicted class: Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot
Prediction probabilities: [[1. 0. 0. 0.]]


In [ ]:
# Load the image you want to predict
img_path = 'RS_Rust 1581.JPG'  # Provide the path to your image
img = image.load_img(img_path, target_size=(250, 250))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array /= 255.  # Rescale pixel values to [0, 1]

# Make prediction
prediction = model.predict(img_array)
predicted_class = np.argmax(prediction)

# Map predicted class index to class label
class_labels = train_generator.class_indices
predicted_label = [k for k, v in class_labels.items() if v == predicted_class][0]

print("Predicted class:", predicted_label)
for i in range(0,4):
    prediction[0][i] = np.round((prediction[0][i]),2)
print("Prediction probabilities:", prediction)


1/1 [==============================] - 0s 26ms/step
Predicted class: Corn_(maize)___Common_rust_
Prediction probabilities: [[0. 1. 0. 0.]]


In [ ]:
# Load the image you want to predict
img_path = 'healthy.jpg'  # Provide the path to your image
img = image.load_img(img_path, target_size=(250, 250))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array /= 255.  # Rescale pixel values to [0, 1]

# Make prediction
prediction = model.predict(img_array)
predicted_class = np.argmax(prediction)

# Map predicted class index to class label
class_labels = train_generator.class_indices
predicted_label = [k for k, v in class_labels.items() if v == predicted_class][0]

print("Predicted class:", predicted_label)
for i in range(0,4):
    prediction[0][i] = np.round((prediction[0][i]),2)
print("Prediction probabilities:", prediction)


1/1 [==============================] - 0s 20ms/step
Predicted class: Corn_(maize)___healthy
Prediction probabilities: [[0. 0. 0. 1.]]


In [ ]:
# Load the image you want to predict
img_path = 'leaf_blight.JPG'  # Provide the path to your image
img = image.load_img(img_path, target_size=(250, 250))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array /= 255.  # Rescale pixel values to [0, 1]

# Make prediction
prediction = model.predict(img_array)
predicted_class = np.argmax(prediction)

# Map predicted class index to class label
class_labels = train_generator.class_indices
predicted_label = [k for k, v in class_labels.items() if v == predicted_class][0]

print("Predicted class:", predicted_label)
for i in range(0,4):
    prediction[0][i] = np.round((prediction[0][i]),2)
print("Prediction probabilities:", prediction)


1/1 [==============================] - 0s 24ms/step
Predicted class: Corn_(maize)___Northern_Leaf_Blight
Prediction probabilities: [[0.06 0.   0.94 0.  ]]
